In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
filepath = "ml-100k/"

ratings = pd.read_csv(f"{filepath}u.data",sep="\t",header = None,names=["user_id", "movie_id", "rating", "timestamp"])

users = pd.read_csv(f"{filepath}u.user",sep="|",header=None,names=["user_id", "age","gender","occupation","zip_code"])

movies = pd.read_csv(f"{filepath}u.item",sep="|",header=None,names=["movie_id","title","release_date","video_release_date","imdb_link","unknown","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film_Noir","Horror","Musical","Mystery","Romance","Sci_Fi","Thriller","War","Western"],encoding='latin-1')
movies = movies.drop("video_release_date",axis=1)

In [12]:
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [38]:
movies.head(10)

,movie_id,title,release_date,imdb_link,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [45]:
users.head(10)

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


## Data Cleaning

In [37]:
movies[movies["title"].duplicated(keep=False)].groupby("title").count()

,movie_id,release_date,imdb_link,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,,
Body Snatchers (1993),2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
"Butcher Boy, The (1998)",2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Chairman of the Board (1998),2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Chasing Amy (1997),2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Deceiver (1997),2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
"Designated Mourner, The (1997)",2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Desperate Measures (1998),2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Fly Away Home (1996),2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Hugo Pool (1997),2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [44]:
movies_no_dupli = movies.drop_duplicates('title')
movies_with_ratings = movies_no_dupli.merge(ratings,on="movie_id")
movies_with_ratings.head(10)

,movie_id,title,release_date,imdb_link,unknown,Action,Adventure,Animation,Children's,Comedy,...,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,user_id,rating,timestamp
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,308,4,887736532
1,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,287,5,875334088
2,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,148,4,877019411
3,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,280,4,891700426
4,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,66,3,883601324
5,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,5,4,875635748
6,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,109,4,880563619
7,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,181,3,878962392
8,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,95,5,879197329
9,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,268,3,875742341


In [46]:
ratings_movies_users = movies_with_ratings.merge(users,on="user_id")

ratings_movies_users.head(10)

,movie_id,title,release_date,imdb_link,unknown,Action,Adventure,Animation,Children's,Comedy,...,Thriller,War,Western,user_id,rating,timestamp,age,gender,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,308,4,887736532,60,M,retired,95076
1,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,287,5,875334088,21,M,salesman,31211
2,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,148,4,877019411,33,M,engineer,97006
3,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,280,4,891700426,30,F,librarian,22903
4,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,66,3,883601324,23,M,student,80521
5,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,5,4,875635748,33,F,other,15213
6,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,109,4,880563619,29,M,other,55423
7,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,181,3,878962392,26,M,executive,21218
8,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,95,5,879197329,31,M,administrator,10707
9,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,268,3,875742341,24,M,engineer,19422


In [47]:
ratings_movies_users_clean = ratings_movies_users.drop(["release_date","imdb_link","occupation","zip_code"],axis=1)

ratings_movies_users_clean.head(10)

,movie_id,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Romance,Sci_Fi,Thriller,War,Western,user_id,rating,timestamp,age,gender
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,308,4,887736532,60,M
1,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,287,5,875334088,21,M
2,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,148,4,877019411,33,M
3,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,280,4,891700426,30,F
4,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,66,3,883601324,23,M
5,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,5,4,875635748,33,F
6,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,109,4,880563619,29,M
7,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,181,3,878962392,26,M
8,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,95,5,879197329,31,M
9,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,268,3,875742341,24,M


In [48]:
ratings_movies_users_clean.isna().sum()

movie_id       0
title          0
unknown        0
Action         0
Adventure      0
Animation      0
Children's     0
Comedy         0
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film_Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci_Fi         0
Thriller       0
War            0
Western        0
user_id        0
rating         0
timestamp      0
age            0
gender         0
dtype: int64

## Model

In [57]:
pivot_table = ratings_movies_users_clean.pivot_table(index = 'title', columns = 'user_id', values = 'rating')
pivot_table.fillna(0, inplace = True)
pivot_table.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-900 (1994),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),5.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187 (1997),0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2 Days in the Valley (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
"20,000 Leagues Under the Sea (1954)",3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),4.0,0.0,0.0,0.0,4.0,5.0,5.0,0.0,0.0,5.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3 Ninjas: High Noon At Mega Mountain (1998),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
scaler = StandardScaler(with_mean=True, with_std=True)
pivot_table_normalized = scaler.fit_transform(pivot_table)
similarity_score = cosine_similarity(pivot_table_normalized)

In [71]:
def recommend(movie_title):
	index = np.where(pivot_table.index == movie_title)[0][0]
	similar_movies = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:6]
	data = []
	for idx,_ in similar_movies:
		movie = []
		temp_df = movies_no_dupli[movies_no_dupli["title"] == pivot_table.index[idx]]
		movie.extend(temp_df["title"].values)
		movie.extend(temp_df["release_date"].values)
		data.append(movie)
	return data

In [73]:
recommend("Toy Story (1995)")

[['Star Wars (1977)', '01-Jan-1977'],
 ['Independence Day (ID4) (1996)', '03-Jul-1996'],
 ['Rock, The (1996)', '07-Jun-1996'],
 ['Return of the Jedi (1983)', '14-Mar-1997'],
 ['Jerry Maguire (1996)', '13-Dec-1996']]